In [1]:
# gpu 확인 -> 1 : 지정됨 / 0 : 지정 안됨

import cv2
cv2.cuda.getCudaEnabledDeviceCount()

1

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math
import datetime as dt
import pandas as pd
from time import strftime
from time import gmtime
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

min_confidence = 0.4 # 사람 검출 bounding box 임계값
vedio_path = './220610_test.mp4' # 사용할 영상 경로
t= 0

def detectAndDisplay(frame):
    # img = cv2.resize(frame, None, fx=0.8, fy=0.8)
    img = frame
    height, width, channels = img.shape

    # 창 크기 설정, input size 지정 가능
    blob = cv2.dnn.blobFromImage(img, 0.00392, (384, 384), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # 탐지한 객체의 클래스 예측 
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            # 원하는 class id 입력 = 'person' / coco.names의 id에서 -1 할 것 
            if class_id == 0 and confidence > min_confidence:
                # 탐지한 객체 boxing
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
               
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
    font = cv2.FONT_HERSHEY_DUPLEX
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = "{}: {:.2f}".format(classes[class_ids[i]], confidences[i]*100)
            color = colors[i] #-- 경계 상자 컬러 설정 / 단일 생상 사용시 (255,255,255)사용(B,G,R)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
    
    return boxes

# yolo 포맷 및 클래스명 불러오기
# model_file = 'yolov3-spp.weights' # 개발 환경에 맞게 변경할 것
# config_file = 'yolov3-spp.cfg' # 개발 환경에 맞게 변경할 것
model_file = 'yolov4-p6.weights'
config_file = 'yolov4-p6.cfg'
net = cv2.dnn.readNet(model_file, config_file)

# GPU 사용
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# 클래스(names파일) 오픈 / 본인 개발 환경에 맞게 변경할 것
classes = []
with open("./coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Video input
cap = cv2.VideoCapture(vedio_path)

if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

res_actual = np.zeros((1,2), dtype=int)# actual resolution of the camera
res_actual[0,0] = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
res_actual[0,1] = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
print("\ncamera resolution: {}".format(res_actual))

# x, y, w, h = rect
x = int(res_actual[0,1]*0.1)
y = int(res_actual[0,0]*0.1)
w = int(res_actual[0,1]*0.8)
h = int(res_actual[0,0]*0.8)

# 필요 변수 선언
i = 0 # frame count & 저장 리스트 비교용
tmp = 0 # 첫번째 detection 구분
notcnt = 0 # 아무것도 검출이 안되는 상황 대비, 일정 카운트 이상일 시 원본 frame 크기로 변경
case_not = 0 # 검출이 안되다가 갑자기 검출될 시 발생하는 예외 제거
event = 0 # 이벤트 상황 구분
j = 1 # 프레임 별 시간 계산용
check = 1 # 자막 append 간격 지정용

# 프레임 간 좌표 계산을 위한 저장용 list 생성, 초기값 append
PX = []
PY = []
X_min = []
X_max = []
Y_min = []
Y_max = []
X_min.append(x)
X_max.append(x+w)
Y_min.append(y)
Y_max.append(y+h)

PX.append((X_max[i] + X_min[i])/2)
PY.append((Y_max[i] + Y_min[i])/2)

# For xlsx
from openpyxl import Workbook
wb = Workbook()
ws = wb.active

with mp_pose.Pose(model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose: # Mediapipe pose setting
    while cap.isOpened():
        success, img = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          break
        
        # 15 fps 인 경우 setting
        t1 = j/15
        
        # 첫번째 detection
        if tmp < 1:
            box = detectAndDisplay(img)
            
            t1 = j/15
            tsame = t1
            
            # (srt) 시작 time 계산
            t1 = format(t1, ".5F")
            start = strftime('%H:%M:%S', gmtime(float(t1)))
            tmp_sec = (int(start[0])*10+int(start[1]))*3600 + (int(start[3])*10+int(start[4])*60 + (int(start[6])*10)+int(start[7]))
            t1_  = format(float(t1)-tmp_sec, ".5F")
            t1_ = pd.to_datetime(t1_, format='%S.%f')
            start_milli = t1_.strftime('%f')
            print("start time :", start, ".", start_milli)
            
            # (srt) 종료 time 계산
            t2 = (j+1)/15
            t2 = format(t2, ".5F")
            end = strftime('%H:%M:%S', gmtime(float(t1)))
            tmp_sec = (int(end[0])*10+int(end[1]))*3600 + (int(end[3])*10+int(end[4])*60 + (int(end[6])*10)+int(end[7]))
            t2_  = format(float(t2)-tmp_sec, ".5F")
            t2_ = pd.to_datetime(t2_, format='%S.%f')
            end_milli = t2_.strftime('%f')
            print("end time :", end, ".", end_milli)
            
            # 사람이 검출될 시
            if len(box) > 0:
                x = box[0][0]
                y = box[0][1]
                w = box[0][2]
                h = box[0][3]
                notcnt = 0
                if case_not > 0:
                    case_not-=1
            elif len(box) == 0 and i % 10 == 0: # 검출 안될 시 image 크기 조금씩 증가, 오래 검출 안될 시 원본 프레임 크기 복구
                x-=1
                y-=1
                w+=2
                h+=2
                notcnt+=1
                if notcnt % 15 == 0:
                    x = int(res_actual[0,1]*0.1)
                    y = int(res_actual[0,0]*0.1)
                    w = int(res_actual[0,1]*0.8)
                    h = int(res_actual[0,0]*0.8)
                    notcnt = 0
                    case_not = 5

            # 임의로 박스 주변 crop 범위 지정
            x_ = int((2*x-w) / 2)
            y_ = int((2*y-h) / 2)
            w_ = int(2*w)
            h_ = int(2*h)
            xw = x_+w_
            yh = y_+h_

            if x_ <= 0: x_ = 0
            if y_ <= 0: y_ = 0
            if xw >= res_actual[0,1]: xw = res_actual[0,1]
            if yh >= res_actual[0,0]: yh = res_actual[0,0]
            
            ##### 낙상 판단 방향
            X_min.append(x)
            X_max.append(x+w)
            Y_min.append(y)
            Y_max.append(y+h)
            
            cv2.imshow('Falling Detection', img)
            
            if len(box) > 0:
                tmp += 1
        
        # crop된 frame 안에서 detection 진행
        elif tmp == 1:
            image = img[y_:yh, x_:xw]
            box = detectAndDisplay(image)
            
            t1 = j/15
            tsame = t1
            
            # (srt) 시작 time 계산
            t1 = format(t1, ".5F")
            start = strftime('%H:%M:%S', gmtime(float(t1)))
            tmp_sec = (int(start[0])*10+int(start[1]))*3600 + (int(start[3])*10+int(start[4])*60 + (int(start[6])*10)+int(start[7]))
            t1_  = format(float(t1)-tmp_sec, ".5F")
            t1_ = pd.to_datetime(t1_, format='%S.%f')
            start_milli = t1_.strftime('%f')
            print("start time :", start, ".", start_milli)
            
            # (srt) 종료 time 계산
            t2 = (j+1)/15
            t2 = format(t2, ".5F")
            end = strftime('%H:%M:%S', gmtime(float(t1)))
            tmp_sec = (int(end[0])*10+int(end[1]))*3600 + (int(end[3])*10+int(end[4])*60 + (int(end[6])*10)+int(end[7]))
            t2_  = format(float(t2)-tmp_sec, ".5F")
            t2_ = pd.to_datetime(t2_, format='%S.%f')
            end_milli = t2_.strftime('%f')
            print("end time :", end, ".", end_milli)
            
            # 축소된 프레임 안에서의 좌표 설정, 조정된 값 계산
            if len(box) > 0:
                x = box[0][0] + x_
                y = box[0][1] + y_
                w = box[0][2]
                h = box[0][3]
                notcnt = 0
                if case_not > 0:
                    case_not-=1
            elif len(box) == 0 and i % 10 == 0: # 검출 안될 시 image 크기 조금씩 증가, 오래 검출 안될 시 원본 프레임 크기 복구
                x-=1
                y-=1
                w+=2
                h+=2
                notcnt+=1
                if notcnt % 15 == 0:
                    x = int(res_actual[0,1]*0.1)
                    y = int(res_actual[0,0]*0.1)
                    w = int(res_actual[0,1]*0.8)
                    h = int(res_actual[0,0]*0.8)
                    case_not = 5
            
            # 임의로 박스 주변 crop 범위 지정
            x_ = int((2*x-w) / 2)
            y_ = int((2*y-h) / 2)
            w_ = int(2*w)
            h_ = int(2*h)
            xw = x_+w_
            yh = y_+h_

            if x_ <= 0: x_ = 0
            if y_ <= 0: y_ = 0
            if xw >= res_actual[0,1]: xw = res_actual[0,1]
            if yh >= res_actual[0,0]: yh = res_actual[0,0]
            
            print("box info :", x,y,x+w,y+h)
            
            # 바운딩 박스 좌표 값 저장
            X_min.append(x)
            X_max.append(x+w)
            Y_min.append(y)
            Y_max.append(y+h)
        
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            # Draw the pose annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            print
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            ### 낙상 판단 방향
            case_d = 0
            if ((X_max[i] + Y_min[i])/2 > (X_min[i] + Y_max[i])/2) and i>0:
                #좌측낙사 case = 3
                if X_min[i] < X_min[i-1] and X_max[i] < X_max[i-1] and Y_min[i] > Y_min[i-1] and Y_max[i] > Y_max[i-1]:
                    case_d = 3

                #우측낙사 case = 3
                elif X_min[i] > X_min[i-1] and X_max[i] > X_max[i-1] and Y_min[i] > Y_min[i-1] and Y_max[i] > Y_max[i-1]:
                    case_d = 3         

                #전방낙사 case = 1
                elif Y_min[i] > Y_min[i-1] and Y_max[i] > Y_max[i-1]:
                    case_d = 1

                #후방낙사 case = 2
                elif Y_min[i] > Y_min[i-1] :
                    case_d = 2

            ### 속력 낙상 판단
            case_v = 0
            PX.append((X_max[i] + X_min[i])/2)
            PY.append((Y_max[i] + Y_min[i])/2)
            Pixel2cm = (Y_max[i-1]+Y_min[i-1])/170

            a = (X_max[i]+X_min[i])/2 - (X_max[i-1]+X_min[i-1])/2
            b = (Y_max[i]+Y_min[i])/2 - (Y_max[i-1]+Y_min[i-1])/2

            #속력 구하기
            V = (math.sqrt(a**2 + b**2) * Pixel2cm)/(1/2)

            if V >= 500 and V < 3000:
                if X_min[i-2] != int(res_actual[0,1]*0.1) and Y_min[i-2] != int(res_actual[0,0]*0.1) and X_max[i-2] != int(res_actual[0,1]*0.9) and Y_max[i-2] != int(res_actual[0,0]*0.9):
                    case_v = 1
            elif V >= 400 and V < 500:
                case_v = 2
            else:
                case_v = 0
            i+=1

            # 33개 keypoint 저장
            if results.pose_landmarks != None:
                landmarks = results.pose_landmarks.landmark
                # 코
                nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y]
                # 왼쪽 발목
                l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                # 오른쪽 발목
                r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                nose[0] = nose[0] * res_actual[0,1]
                nose[1] = nose[1] * res_actual[0,0]
                l_ankle[0] = l_ankle[0] * res_actual[0,1]
                l_ankle[1] = l_ankle[1] * res_actual[0,0]
                r_ankle[0] = r_ankle[0] * res_actual[0,1]
                r_ankle[1] = r_ankle[1] * res_actual[0,0]

                # 1번, 얼굴 좌표(코)의 y값이 하단 좌표(발목 중점)의 y값보다 아래 위치
                # 현재는 사용하지 않음(kepoint 좌표 불안정 문제)
                case_s = 0
                if nose[1] > (l_ankle[1] + r_ankle[1]) / 2:
                    case_s = 1
                else:
                    case_s = 0
            
            # 정상일 때 자막 저장 & 이상 상황 발생 시 event 값 부여
            if event == 0:
                if case_not == 0 and case_d == 1 and case_v == 1: # and case_s == 1:
                    event = 1
                elif case_not == 0 and case_d == 2 and case_v == 1:
                    event = 2
                elif case_not == 0 and case_d == 3 and case_v == 1:
                    event = 3
                elif case_not == 0 and case_d > 0 and case_v == 2:
                    event = 4
                else:
                    print("<<<정상>>>")
                    print("속도 :", V)
                    if check % 15 == 0:
                        ws.append(['정상', start, start_milli, end, end_milli])
            
            # 전방 낙상 경우
            elif event == 1:
                print("<<<앞으로 넘어짐!>>>")
                print("속도 :", V)
                if check % 15 == 0:
                    ws.append(['앞으로 넘어짐!', start, start_milli, end, end_milli])
            
            # 후방 낙상 경우
            elif event == 2:
                print("<<<뒤로 넘어짐!>>>")
                print("속도 :", V)
                if check % 15 == 0:
                    ws.append(['뒤로 넘어짐!', start, start_milli, end, end_milli])
                    
            # 측면 낙상 경우
            elif event == 3:
                print("<<<옆으로 넘어짐!>>>")
                print("속도 :", V)
                if check % 15 == 0:
                    ws.append(['옆으로 넘어짐!', start, start_milli, end, end_milli])
            
            # 낙상이 의심되는 상황
            elif event == 4:
                print("<<<낙상 의심>>>")
                print("속도 :", V)
                if check % 15 == 0:
                    ws.append(['낙상 의심', start, start_milli, end, end_milli])
                    event = 0

            j+=1
            check+=1
            
            # 결과 show
            cv2.imshow('Falling Detection', image)

            key_pressed = cv2.waitKey(1) & 0xFF
            if key_pressed == ord('q'):
                break

        print("--------------------------------------------------------")
                
wb.save('log.xlsx') # 로그 저장

cap.release()
cv2.destroyAllWindows()


camera resolution: [[1080 1920]]
start time : 00:00:00 . 066670
end time : 00:00:00 . 133330
--------------------------------------------------------
start time : 00:00:00 . 066670
end time : 00:00:00 . 133330
box info : 588 342 877 720
<<<정상>>>
속도 : 2844.635122678561
--------------------------------------------------------
start time : 00:00:00 . 133330
end time : 00:00:00 . 200000
box info : 586 339 877 726
<<<정상>>>
속도 : 2816.7898094195502
--------------------------------------------------------
start time : 00:00:00 . 200000
end time : 00:00:00 . 266670
box info : 589 347 884 692
<<<정상>>>
속도 : 131.55280361287745
--------------------------------------------------------
start time : 00:00:00 . 266670
end time : 00:00:00 . 333330
box info : 587 343 874 729
<<<정상>>>
속도 : 22.52409090907504
--------------------------------------------------------
start time : 00:00:00 . 333330
end time : 00:00:00 . 400000
box info : 588 350 882 690
<<<정상>>>
속도 : 174.5145119435422
------------------------

--------------------------------------------------------
start time : 00:00:03 . 266670
end time : 00:00:03 . 333330
box info : 490 447 862 716
<<<정상>>>
속도 : 69.34414178573414
--------------------------------------------------------
start time : 00:00:03 . 333330
end time : 00:00:03 . 400000
box info : 488 461 845 716
<<<정상>>>
속도 : 15.442022385792667
--------------------------------------------------------
start time : 00:00:03 . 400000
end time : 00:00:03 . 466670
box info : 497 455 837 710
<<<정상>>>
속도 : 215.93010403134997
--------------------------------------------------------
start time : 00:00:03 . 466670
end time : 00:00:03 . 533330
box info : 520 451 843 717
<<<정상>>>
속도 : 161.45756220897078
--------------------------------------------------------
start time : 00:00:03 . 533330
end time : 00:00:03 . 600000
box info : 555 449 834 717
<<<정상>>>
속도 : 83.3703342308149
--------------------------------------------------------
start time : 00:00:03 . 600000
end time : 00:00:03 . 666670
b

start time : 00:00:06 . 466670
end time : 00:00:06 . 533330
box info : 581 335 878 727
<<<정상>>>
속도 : 14.021461553028093
--------------------------------------------------------
start time : 00:00:06 . 533330
end time : 00:00:06 . 600000
box info : 580 328 879 726
<<<정상>>>
속도 : 138.67517621503097
--------------------------------------------------------
start time : 00:00:06 . 600000
end time : 00:00:06 . 666670
box info : 579 324 879 727
<<<정상>>>
속도 : 123.5887924505178
--------------------------------------------------------
start time : 00:00:06 . 666670
end time : 00:00:06 . 733330
box info : 578 325 880 727
<<<정상>>>
속도 : 49.976470588235294
--------------------------------------------------------
start time : 00:00:06 . 733330
end time : 00:00:06 . 800000
box info : 569 341 879 701
<<<정상>>>
속도 : 19.606121493043954
--------------------------------------------------------
start time : 00:00:06 . 800000
end time : 00:00:06 . 866670
box info : 580 329 880 724
<<<정상>>>
속도 : 6.1823529411764

start time : 00:00:09 . 666670
end time : 00:00:09 . 733330
box info : 583 326 879 727
<<<정상>>>
속도 : 107.35376712011038
--------------------------------------------------------
start time : 00:00:09 . 733330
end time : 00:00:09 . 800000
box info : 579 321 879 727
<<<정상>>>
속도 : 117.52685874280635
--------------------------------------------------------
start time : 00:00:09 . 800000
end time : 00:00:09 . 866670
box info : 572 337 877 702
<<<정상>>>
속도 : 52.52049361092247
--------------------------------------------------------
start time : 00:00:09 . 866670
end time : 00:00:09 . 933330
box info : 581 325 878 729
<<<정상>>>
속도 : 39.66170483539288
--------------------------------------------------------
start time : 00:00:09 . 933330
end time : 00:00:09 . 000000
box info : 579 321 878 726
<<<정상>>>
속도 : 78.46389600178254
--------------------------------------------------------
start time : 00:00:10 . 000000
end time : 00:00:10 . 066670
box info : 575 338 874 697
<<<정상>>>
속도 : 110.1814051531495

start time : 00:00:12 . 800000
end time : 00:00:12 . 866670
box info : 572 337 878 698
<<<정상>>>
속도 : 101.0703369460131
--------------------------------------------------------
start time : 00:00:12 . 866670
end time : 00:00:12 . 933330
box info : 582 321 879 728
<<<정상>>>
속도 : 27.70093623890916
--------------------------------------------------------
start time : 00:00:12 . 933330
end time : 00:00:12 . 000000
box info : 575 338 877 702
<<<정상>>>
속도 : 96.58498328694681
--------------------------------------------------------
start time : 00:00:13 . 000000
end time : 00:00:13 . 066670
box info : 582 328 879 730
<<<정상>>>
속도 : 108.39794763695075
--------------------------------------------------------
start time : 00:00:13 . 066670
end time : 00:00:13 . 133330
box info : 573 340 877 701
<<<정상>>>
속도 : 78.53876613155524
--------------------------------------------------------
start time : 00:00:13 . 133330
end time : 00:00:13 . 200000
box info : 582 324 878 730
<<<정상>>>
속도 : 123.1152721729296


...

start time : 00:02:44 . 400000
end time : 00:02:44 . 466670
box info : 536 623 759 961
<<<옆으로 넘어짐!>>>
속도 : 363.06652578464
--------------------------------------------------------
start time : 00:02:44 . 466670
end time : 00:02:44 . 533330
box info : 534 663 758 959
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:44 . 533330
end time : 00:02:44 . 600000
box info : 536 664 759 958
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:44 . 600000
end time : 00:02:44 . 666670
box info : 537 664 759 959
<<<옆으로 넘어짐!>>>
속도 : 355.1722798497969
--------------------------------------------------------
start time : 00:02:44 . 666670
end time : 00:02:44 . 733330
box info : 537 664 759 959
<<<옆으로 넘어짐!>>>
속도 : 28.623529411764707
--------------------------------------------------------
start time : 00:02:44 . 733330
end time : 00:02:44 . 800000
box info : 537 664 759 959
<<<옆으로 넘어짐!>>>
속도 : 13.493261165

start time : 00:02:47 . 666670
end time : 00:02:47 . 733330
box info : 531 602 763 960
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:47 . 733330
end time : 00:02:47 . 800000
box info : 531 602 763 960
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:47 . 800000
end time : 00:02:47 . 866670
box info : 531 602 763 960
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:47 . 866670
end time : 00:02:47 . 933330
box info : 531 602 763 960
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:47 . 933330
end time : 00:02:47 . 000000
box info : 531 602 763 960
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:48 . 000000
end time : 00:02:48 . 066670
box info : 531 602 763 960
<<<옆으로 넘어짐!>>>
속도 : 0.0
-------------------------------------------------

start time : 00:02:50 . 866670
end time : 00:02:50 . 933330
box info : 533 607 759 961
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:50 . 933330
end time : 00:02:50 . 000000
box info : 533 607 759 961
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:51 . 000000
end time : 00:02:51 . 066670
box info : 535 660 758 955
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:51 . 066670
end time : 00:02:51 . 133330
box info : 533 609 758 959
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:51 . 133330
end time : 00:02:51 . 200000
box info : 533 609 758 959
<<<옆으로 넘어짐!>>>
속도 : 433.6039939038982
--------------------------------------------------------
start time : 00:02:51 . 200000
end time : 00:02:51 . 266670
box info : 533 609 758 959
<<<옆으로 넘어짐!>>>
속도 : 446.9040724808849
---------------------

start time : 00:02:54 . 133330
end time : 00:02:54 . 200000
box info : 535 609 761 961
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:54 . 200000
end time : 00:02:54 . 266670
box info : 535 609 761 961
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:54 . 266670
end time : 00:02:54 . 333330
box info : 535 609 761 961
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:54 . 333330
end time : 00:02:54 . 400000
box info : 535 609 761 961
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:54 . 400000
end time : 00:02:54 . 466670
box info : 535 609 761 961
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:54 . 466670
end time : 00:02:54 . 533330
box info : 535 609 761 961
<<<옆으로 넘어짐!>>>
속도 : 0.0
-------------------------------------------------

start time : 00:02:57 . 400000
end time : 00:02:57 . 466670
box info : 530 604 766 966
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:57 . 466670
end time : 00:02:57 . 533330
box info : 530 604 766 966
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:57 . 533330
end time : 00:02:57 . 600000
box info : 530 604 766 966
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:57 . 600000
end time : 00:02:57 . 666670
box info : 530 604 766 966
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:57 . 666670
end time : 00:02:57 . 733330
box info : 530 604 766 966
<<<옆으로 넘어짐!>>>
속도 : 0.0
--------------------------------------------------------
start time : 00:02:57 . 733330
end time : 00:02:57 . 800000
box info : 535 663 758 958
<<<옆으로 넘어짐!>>>
속도 : 0.0
-------------------------------------------------

In [3]:
## xlsx to srt (srt 만들기)
import pandas as pd

column_names = ["text", "start", "start_milli", "end", "end_milli"]
input_file = pd.ExcelFile("log.xlsx")
sheet1 = input_file.parse(0, header=None, names=column_names)
counter = 1
with open("./220610_test.srt", 'w') as file:
    for index, row in sheet1.iterrows():
        milli1 = str(row["start_milli"]).split(".")[0].replace("nan", "000")
        milli2 = str(row["end_milli"]).split(".")[0].replace("nan", "000")
        print("%d\n%s,%s --> %s,%s\n%s\n" % (counter, row["start"], milli1, row["end"], milli2, row["text"]), file=file)
        counter += 1